In [ ]:
# Tutorial 101_FileIO.py

import sys, os
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

debug = False

# Load a mesh in OFF format
v, f, _ = igl.readOFF(igl.tutorial_path + "cube.off", read_normals=True)

# Print the vertices and faces matrices 
if debug:
    print("Vertices: \n", v, sep='')
    print("Faces: \n", f, sep='')

# Save the mesh in OBJ format
ret = igl.writeOBJ(igl.tutorial_path + "cube.obj", v, f)
if debug:
    print(ret)

In [ ]:
# Tutorial 102_DrawMesh.py

import sys, os
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

# Load a mesh in OFF format
v, f, _ = igl.readOFF(igl.tutorial_path + "bunny.off", read_normals=False)

# Plot the mesh
viewer = igl.Viewer()
viewer.set_mesh(v, f)
viewer.launch()

In [ ]:
# Tutorial 103_Events.py

import sys, os
import ipywidgets as widgets
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

def select(change):
    if change["new"] == 1:
        #viewer.data().clear()
        viewer.set_mesh(v1, f1)
        #viewer.core.align_camera_center(V1,F1)
    if change["new"] == 2:
        viewer.set_mesh(v2, f2)

#  Load two meshes
v1, f1, _ = igl.readOFF(igl.tutorial_path + "bumpy.off", read_normals=False)
v2, f2, _ = igl.readOFF(igl.tutorial_path + "fertility.off",read_normals=False)

viewer = igl.Viewer()

# Create a widget and assign callback to switch between the two loaded meshes
viewer.add_widget(widgets.Dropdown(
    options={'Bump Mesh': 1, 'Fertility Mesh': 2},
    value=1,
    description='Select Mesh',
), select)

viewer.set_mesh(v1, f1)
viewer.launch()

In [ ]:
# Tutorial 104_Colors.py

import sys, os
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

# Load a mesh in OFF format
v, f, _ = igl.readOFF(igl.tutorial_path + "screwdriver.off")

# Use the z coordinate as a scalar field over the surface
z = v[:, 2]
c = igl.jet(z, True) # JET is considered evil, use better colormap instead

# Alternative numpy/matplotlib colorpalettes
#import matplotlib.pyplot as plt
#import numpy as np

#def get_colors(inp, colormap, vmin=None, vmax=None):
#    norm = plt.Normalize(vmin, vmax)
#    return colormap(norm(inp))
#c = get_colors(z, plt.cm.viridis, vmin=np.min(z), vmax=np.max(z))

# Plot the mesh
viewer = igl.Viewer()
viewer.set_mesh(v, f)#, c)

# Add per-vertex colors
viewer.set_colors(c)
# viewer.data().set_colors(c)

# Launch the viewer
viewer.launch()

In [ ]:
# Tutorial 105_Overlays.py

import sys, os
import numpy as np
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

# Load a mesh in OFF format
v, f, _ = igl.readOFF(igl.tutorial_path + "bunny.off")

# Find the bounding box
m = np.min(v, axis=0)
M = np.max(v, axis=0)

# Corners of the bounding box
v_box = np.array(
    [
        [m[0], m[1], m[2]],
        [M[0], m[1], m[2]],
        [M[0], M[1], m[2]],
        [m[0], M[1], m[2]],
        [m[0], m[1], M[2]],
        [M[0], m[1], M[2]],
        [M[0], M[1], M[2]],
        [m[0], M[1], M[2]]
    ]
)

f_box = np.array(
        [
            [0, 1],
            [1, 2],
            [2, 3],
            [3, 0],
            [4, 5],
            [5, 6],
            [6, 7],
            [7, 4],
            [0, 4],
            [1, 5],
            [2, 6],
            [7, 3]
        ]
, dtype=np.int)

# Plot the mesh
viewer = igl.Viewer()
viewer.set_mesh(v, f)

# Plot the corners of the bounding box as points
# viewer.data().add_points(V_box, igl.eigen.MatrixXd([[1, 0, 0]]))

# Plot the edges of the bounding box
#for i in range(0, E_box.rows()):
#    viewer.data().add_edges(
#        V_box.row(E_box[i, 0]),
#        V_box.row(E_box[i, 1]),
#        igl.eigen.MatrixXd([[1, 0, 0]]))

# Plot labels with the coordinates of bounding box vertices
#l1 = 'x: ' + str(m[0, 0]) + ' y: ' + str(m[0, 1]) + ' z: ' + str(m[0, 2])
#viewer.data().add_label(m.transpose(), l1)

#l2 = 'x: ' + str(M[0, 0]) + ' y: ' + str(M[0, 1]) + ' z: ' + str(M[0, 2])
#viewer.data().add_label(M.transpose(), l2)

# Launch the viewer
viewer.launch()

In [ ]:
# Tutorial 106_ViewerMenu.py

# Needs support for GUI elements

In [ ]:
# Tutorial 107_MultipleMeshes.py

# Needs refactoring of data element in viewer. Idea is to store a list of data which is rendered in the viewer

In [ ]:
# Tutorial 201_Normals.py

# Needs different normal rendering support in Viewer

In [ ]:
# Tutorial 202_GaussianCurvature.py

import sys, os
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl

# Load mesh
v, f, _ = igl.readOFF(igl.tutorial_path + "bumpy.off")

# Compute Gaussian curvature
k = igl.gaussian_curvature(v, f)

# Compute pseudocolor
c = igl.jet(k, True)

# Plot the mesh with pseudocolors
viewer = igl.Viewer()
viewer.set_mesh(v, f)
viewer.set_colors(c)
viewer.launch()

In [ ]:
# Tutorial 604_Triangle.py

import sys, os

import sys, os
# Add the igl library to the modules search path
sys.path.insert(0, os.getcwd() + "/../")
import igl
import numpy as np

# Input polygon
v = np.array([[-1.0, -1.0], [1, -1], [1, 1], [-1, 1], [-2, -2], [2, -2], [2, 2], [-2, 2]])
e = np.array([[0, 1], [1, 2], [2, 3], [3, 0], [4, 5], [5, 6], [6,7], [7,4]], dtype=np.int32)
h = np.array([[0.0, 0.0]])

v, f = igl.triangulate(v, e, h, flags="a0.005q")
ext = np.zeros((v.shape[0],1))
v = np.append(v, ext, axis=1)

# Plot the mesh
viewer = igl.Viewer()
viewer.set_mesh(v, f)
viewer.launch()

In [ ]:
from pythreejs import *
from IPython.display import display

class Vieweri():
    def __init__(self):
        #print("Viewer initialization")
        self.data = {}
        self.cam = PerspectiveCamera(position=[0, 0, 1], lookAt=[0, 0, 0],fov=30,
                        children=[DirectionalLight(color='white', position=[0, 0, 1], intensity=0.6)])
        self.renderer = Renderer(camera=self.cam, scene = Scene(), controls=[OrbitControls(controlling=self.cam)], 
                    width=600, height=600, antialias=False)
        self.widgets = []

    def set_mesh(self, v, f, c=None):
        v = v.astype("float32", copy=False)
        f = f.astype("uint16", copy=False).ravel()
        
        if type(c) == type(None):
            c = np.ones_like(v)
            c[:, 1] = 0.874
            c[:, 2] = 0.0
            geometry = BufferGeometry(attributes=dict(
                position=BufferAttribute(v, normalized=False),
                index=BufferAttribute(f, normalized=False),
                color=BufferAttribute(c)
            ))
        else:
            c = c.astype("float32", copy=False)
            c = c[:, :3]
            geometry = BufferGeometry(attributes=dict(
                position=BufferAttribute(v, normalized=False),
                index=BufferAttribute(f, normalized=False),
                color=BufferAttribute(c),
            ))
        geometry.exec_three_obj_method('computeVertexNormals')
        self.mesh = Mesh(geometry=geometry, material=[MeshStandardMaterial(vertexColors='VertexColors', reflectivity=1.0, 
                                                            #side='FrontSide', 
                                                            roughness=0.5, metalness=0.25, flatShading=False,
                                                            polygonOffset=True, polygonOffsetFactor= 1,
                                                            polygonOffsetUnits=1)])
#MeshLambertMaterial(vertexColors='VertexColors')
        geon = WireframeGeometry(self.mesh.geometry) # WireframeGeometry
        mat = LineBasicMaterial(color="black", linewidth=0.6)
        wireframe = LineSegments( geon, mat )
        self.mesh.add(wireframe)

        self.scene = Scene(children=[self.mesh, self.cam, AmbientLight(intensity=0.5)], background="#4c4c80")
        self.renderer.scene = self.scene
        
    def set_colors(self, c):
        c = c.astype("float32", copy=False)
        c = c[:, :3]
        self.mesh.geometry.attributes.get("color").array = c
        
    def add_lines(self, lines):#, colors):
        #mat = LineBasicMaterial(color="black", linewidth=0.6)
        linesgeom = Geometry(vertices=lines)#, colors = colors)# ['red', 'red', 'green', 'green', 'white', 'orange'])
        lines = LineSegments(geometry=linesgeom, 
             material=LineBasicMaterial(linewidth=5, color="red"))#,#vertexColors='VertexColors'), 
             #type='LinePieces')
        self.mesh.add(lines)
    
    def add_arrows(self, arrow, length):#, colors):
        #mat = LineBasicMaterial(color="black", linewidth=0.6)
        #linesgeom = Geometry(vertices=lines)#, colors = colors)# ['red', 'red', 'green', 'green', 'white', 'orange'])
        #lines = LineSegments(geometry=linesgeom, 
             #material=LineBasicMaterial(linewidth=5, color="red"))#,#vertexColors='VertexColors'), 
             #type='LinePieces')
        arrowhelper = ArrowHelper(dir=arrow[1], origin=arrow[0], length=length);
        self.mesh.add(arrowhelper)
        
    def add_widget(self, widget, callback):
        self.widgets.append(widget)
        widget.observe(callback, names='value')

    def launch(self):
        display(self.renderer)
        for w in self.widgets:
            display(w)


In [ ]:
from pynput import keyboard

def on_press(key):
    try:
        print('alphanumeric key {0} pressed'.format(
            key.char))
    except AttributeError:
        print('special key {0} pressed'.format(
            key))

def on_release(key):
    print('{0} released'.format(
        key))
    if key == keyboard.Key.esc:
        # Stop listener
        return False

# Collect events until released
with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
    listener.join()

In [ ]:
a